In [ ]:
import h5py
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
import os
import cv2
import ast
from scipy.spatial.transform import Rotation as R
from tqdm import tqdm
from PIL import Image

In [ ]:
dir_with_hdf5_datasets = '/disks/hdd6/datasets/Habitat/train_depth_0_100/dataset_hdf5'

In [ ]:
filename_hdf5_dataset_list = os.listdir(dir_with_hdf5_datasets)

In [ ]:
T = np.array(dtype=np.float32, object=[
    [1, 0, 0, 0],
    [0, -1, 0, 0],
    [1, 0, -1, 0],
    [0, 0, 0, 1]
])
# T = np.eye(4)
T_inv = np.linalg.inv(T)
r = R.from_rotvec(np.pi * np.array([1, 0, 0]))

In [ ]:
def action_to_mat44(action, rot_delta, pos_delta):
    if action == 0:
        a = 0
        d = 0
    if action == 1:
        a = 0
        d = pos_delta
    elif action == 2:
        a = -rot_delta
        d = 0
    elif action == 3:
        a = rot_delta
        d = 0
    a = np.pi * a / 180
    d /= 100
    T = np.array([
        [np.cos(a), 0, np.sin(a), 0],
        [0, 1, 0, 0],
        [-np.sin(a), 0, np.cos(a), d],
        [0, 0, 0, 1]
    ])
    return T

In [ ]:
for filename_hdf5_dataset in tqdm(filename_hdf5_dataset_list):
    rot_delta, pos_delta = map(float, filename_hdf5_dataset.split('_')[1:3])
    
    file = h5py.File(os.path.join(dir_with_hdf5_datasets, filename_hdf5_dataset), 'r')
    output_path = dir_with_hdf5_datasets[:dir_with_hdf5_datasets.rfind('/')]+'/temp_datasets/'+ \
                  filename_hdf5_dataset[:filename_hdf5_dataset.rfind('.')]
    
    rgb = file['rgb']
    depth = file['depth']
    semantic = file['semantic']
    objectgoal = file['objectgoal']
    compass = file['compass']
    gps = file['gps']
    heading = file['heading']
    top_down_map = file['top_down_map']
    fog_of_war_mask = file['fog_of_war_mask']
    agent_map_coord = file['agent_map_coord']
    agent_angle = file['agent_angle']
    mapping = np.array(file['mapping'])
    index_to_title_map = ast.literal_eval(str(np.array(file['index_to_title_map']))[2:-1])
    positions = file['position']
    rotations = file['rotation']
    actions = file['action']
    
    os.makedirs(os.path.join(output_path,'rgb'), exist_ok=True)
    os.makedirs(os.path.join(output_path,'depth'), exist_ok=True)
    depth_file = open(os.path.join(output_path, 'depth.txt'), 'w')
    rgb_file = open(os.path.join(output_path, 'rgb.txt'), 'w')
    gt_filename = os.path.join('/disks/hdd6/datasets/Habitat/train_depth_0_100/temp_gt',filename_hdf5_dataset.split('/')[-1])
    gt_filename = gt_filename[:gt_filename.rfind('.')] + '.txt'
    gt_file = open(gt_filename, 'w')
    rgb_file.close()
    depth_file.close()
    gt_file.close()
    depth_file = open(os.path.join(output_path, 'depth.txt'), 'a')
    rgb_file = open(os.path.join(output_path, 'rgb.txt'), 'a')
    gt_file = open(gt_filename, 'a')
    action_file = open(os.path.join(output_path, 'control.txt'), 'w')
    for i in range(3):
        depth_file.write('ignore\n')
        rgb_file.write('ignore\n')
    for i in range(0, len(rgb)):
        cv2.imwrite(os.path.join(output_path, 'rgb', str(i).zfill(6)+'.png'), rgb[i])
        
        depth_image = 100 * depth[i]
        
        cv2.imwrite(os.path.join(output_path, 'depth', str(i).zfill(6)+'.exr'), depth_image)
        rgb_file.write('{:.3f} '.format(i/10))
        rgb_file.write('rgb/'+str(i).zfill(6)+'.png\n')
        depth_file.write('{:.3f} '.format(i/10))
        depth_file.write('depth/'+str(i).zfill(6)+'.exr\n')
        
        X, Y, Z = positions[i]
        w, x, y, z = rotations[i]
        
        x, y, z, w = (R.from_quat([x, y, z, w]) * r).as_quat()
        
        gt_file.write('{:.3f} {:.8e} {:.8e} {:.8e} {:.8e} {:.8e} {:.8e} {:.8e}\n'\
              .format(i/10, X, Y, Z, x, y, z, w))
        
        action_file.write('{:.3f} '.format(i/10))
        action_file.write(('{:.8e} ' * 12 + '\n').format(*action_to_mat44(actions[i - 1], rot_delta, pos_delta)[:3].flatten()))
    
    rgb_file.close()
    depth_file.close()
    gt_file.close()
    action_file.close()